In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

ModuleNotFoundError: No module named 'ipywidgets'

In [23]:
#Instalar SURPRISE en Anaconda Prompt
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [24]:
conn=sql.connect('db_movies')
cur=conn.cursor() 

Sistema de recomendación basado en contenido KNN con base en todo lo visto por el usuario

In [25]:
movies=pd.read_sql('select * from moviesready',conn)
movies['año_estreno']=movies.año_estreno.astype('int')

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      2121 non-null   int64 
 1   title        2121 non-null   object
 2   cnt_rat      2121 non-null   int64 
 3   Action       2121 non-null   int64 
 4   Adventure    2121 non-null   int64 
 5   Animation    2121 non-null   int64 
 6   Children     2121 non-null   int64 
 7   Comedy       2121 non-null   int64 
 8   Crime        2121 non-null   int64 
 9   Documentary  2121 non-null   int64 
 10  Drama        2121 non-null   int64 
 11  Fantasy      2121 non-null   int64 
 12  Film-Noir    2121 non-null   int64 
 13  Horror       2121 non-null   int64 
 14  IMAX         2121 non-null   int64 
 15  Musical      2121 non-null   int64 
 16  Mystery      2121 non-null   int64 
 17  Romance      2121 non-null   int64 
 18  Sci-Fi       2121 non-null   int64 
 19  Thriller     2121 non-null 

In [26]:
##### cargar data frame escalado y con dummies ###

movies_dum1= joblib.load('salidas_movies_dum1.joblib')

usuarios=pd.read_sql('select distinct (user_id) as user_id from ratingsready',conn)

user_id=89

In [50]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ratings=pd.read_sql('select *from ratingsready where user_id=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movieid'].to_numpy()
    
    ####agregar la columna de movie_id y title de la pelicula a dummie para filtrar y mostrar nombre
    movies_dum1[['movie_id','title']]=movies[['movieId','title']]

    ### filtrar peliculas calificadas por el usuario
    movies_r=movies_dum1[movies_dum1['movie_id'].isin(l_movies_r)]

    ## eliminar columna movie_id' y title
    movies_r=movies_r.drop(columns=['movie_id','title'])
    movies_r["indice"]=1 
        
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean() 

    ### filtrar peliculas no vistas    
    movies_nr=movies_dum1[~movies_dum1['movie_id'].isin(l_movies_r)] 
        
    ## eliminar movie_id y title
    movies_nr=movies_nr.drop(columns=['movie_id','title'])

    ### entrenar modelo 
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr) 
    dist, idlist = model.kneighbors(centroide)
        
    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]
    leidos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]
    return recomend_b

recomendar(30)

print(interact(recomendar))


interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x0000021954930FE0>
